The next step in giving my project legs was to actually get data into my analysis stack. For this project, I planned on using a pandas/scikit-learn/statsmodels stack.

To this end, I needed to get a hold of the code dictionary available on this webpage, and use it to rewrite the 2013 MEPS dataset as a comma separated values file.

The first thing to do is to instantiate a scraper using the BeautifulSoup and requests libraries.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
matplotlib.pylab.rcParams['figure.figsize'] = (10, 6)

I may run this analysis on several MEPS datasets in the future, so I'm going to hardcode a number of pages into a dictionary for easy access later. Here's a function that accesses the dictionary using project names.

In [2]:
def theurl(name):
    answer={"2012":"http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H155", 
            "2013":"http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H163", 
            "population": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H157", 
            "medical": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H154", 
            "risk": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H140", 
            "employment": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H131", 
            "jobs": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H158", 
            "person_round_plan": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H153"}
    return answer[name]

So for instance, calling it on "2013" yields:

In [3]:
theurl("2013")

'http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H163'

Let's write a function that returns the parse tree for the html we need:

In [4]:
def make_soup(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.content)
    return soup

And let's check that this worked for our page:

In [5]:
this=make_soup(theurl("2013"))

In [6]:
this.title

<title>Medical Expenditure Panel Survey PUF Codebook</title>

I fiddled around with this object for a while and discovered that the data I needed all lay inside bits tagged "font".

In [7]:
this.findAll('font',limit=60)

[<font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font color="#660000">::</font>,
 <font class="smallBlack" face="Arial">MEPS HC-163</font>,
 <font class="smallBlack" face="Arial">2013 FULL YEAR CONSOLIDATED DATA CODEBOOK</font>,
 <font class="smallBlack" face="Arial">DATE:   October 7, 2015</font>,
 <font class="smallBlack" face="Arial">940   </font>,
 <font class="smallBlack" face="Arial">941   </font>,
 <font class="smallBlack" face="Arial">PERS ELIGIBLE FOR ACCE

These tagged pieces contain the variable description, preceded by the start and stop columns for the position of the field in the fixed-width file.

For example:

    <font class="smallBlack" face="Arial">353   </font>,
    <font class="smallBlack" face="Arial">354   </font>,
    <font class="smallBlack" face="Arial">CANCER DIAGNOSED - MELANOMA (&gt;17)</font>,

The next function extracts all the tags in the parse tree with attribute "font" and collects them in a list, that I (in very poor taste) called final. In my defense, the list is really nothing, and nothing is ever final, so there you go.

I then use a helper function I called "prune_list" to get rid of the crud I didn't want to deal with that also happened to be part of a "font" tag. Finally, this function writes the code dictionary table to the file provided as the second argument to the function.

In [8]:
def get_data(soup,target):
    result=[]
    for row in soup.find_all("font"):
        result.append(row.contents)
        final=[(term[0].replace(u'\xa0',u'')).replace(',','') for term in result]
        #get rid of junk
        theterms=["::",
                  "MEPS HC-155",
                  "MEPS HC-163",
                  "MEPS HC-157",
                  "MEPS H154 CODEBOOK",
                  "MEPS H140 CODEBOOK",
                  "MEPS H131 CODEBOOK",
                  "MEPS HC-150",
                  "MEPS HC-158",
                  "MEPS H153 CODEBOOK",
                  "2012 FULL YEAR CONSOLIDATED DATA CODEBOOK",
                  "2013 FULL YEAR CONSOLIDATED DATA CODEBOOK",
                  "2013 FULL YEAR POPULATION CHARACTERISTICS CODEBOOK",
                  "2012 MEPS MEDICAL CONDITIONS FILE",
                  "2002-2009 RISK ADJUSTMENT SCORES FILE",
                  "EMPLOYMENT VARIABLE IMPUTATION FILE",
                  "2012 JOBS FILE CODEBOOK",
                  "2013 JOBS FILE CODEBOOK",
                  "2012 PERSON ROUND PLAN FILE",
                  "DATE:   August 25 2014",
                  "DATE:   August 25 2015",
                  "DATE: August 21 2014",
                  "DATE: August 4 2014",
                  "DATE:     March 6 2015",
                  "DATE: December 15 2014",
                  "DATE:    April 10 2013",
                  "DATE:   August 12 2014",
                  "DATE: February 13 2015"]                          
        final=prune_list(final,theterms)
        with open(target,"w") as f:
            f.write("start,end,variable\n")
            for skip in range(0,len(final)-1,3):
                f.write("%s,"%final[skip])
                f.write("%s,"%final[skip+1])
                f.write("%s\n"%((final[skip+2]).lstrip()).rstrip())
        print("Done")
        temp=pd.read_csv(target)
        temp=temp.sort("start").copy()
        temp.index=list(range(len(temp)))
        return temp

Here's the definition of prune_list for reference:

In [9]:
def prune_list(thelist, theterms):
    for theterm in theterms:
        while True:
            try:
                thelist.pop(thelist.index(theterm))
            except:
                break
    return thelist

Finally, here's the function that takes the MEPS dataset, and a dataframe containing the column dictionary, and writes the dataset out into a comma separated values file:

In [10]:
def write_table(data,header,target,short="No"):
    if short!="No":
        data=data[:10]
    data=prune_list(data,'')
    header=header.sort("start").copy()
    header.index=list(range(len(header)))
    with open(target,"w") as f:
        for element in header["variable"]:
            f.write("%s, "%element)
        f.write("\n")
    for row in data:
        u=[row[(header["start"].iloc[place]-1):(header["end"].iloc[place])] for place in header.index]
        with open(target,"a") as f:
            writer=csv.writer(f)
            writer.writerow(u)
            #print("Done with row %s"%data.index(row))
